In [ ]:
import netCDF4 as nc
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import os
import math
#import imageio
from scipy.ndimage.interpolation import map_coordinates
from pprint import pprint
import glob
from scipy import interpolate
from scipy.interpolate import RegularGridInterpolator
from copy import copy
from copy import deepcopy
#from mpl_toolkits.axes_grid1 import make
#_axes_locatable
%matplotlib inline
from scipy import stats
import multiprocessing as mp
import matplotlib.patches as patches


In [ ]:
tsc = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2001/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
lifetime = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2001/*.nc',combine='nested',concat_dim='time')['Lifetime'][:,2:62]
landocean = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2001/*.nc',combine='nested',concat_dim='time')['ConvOrigin'][:,2:62]
detinsit = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2001/*.nc',combine='nested',concat_dim='time')['ConvCir'][:,2:62]
pc = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2001/*.nc',combine='nested',concat_dim='time')['ConvPc'][:,2:62]


In [ ]:
lifetime = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/19*/*.nc',combine='nested',concat_dim='time')['Lifetime'][:,2:62]

In [ ]:
lifetime_ya = []
tsc_ya = []
oceanavg = []
landavg = []
for i in range(1984,2017,1):
    try:
        dataset = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/'+str(i)+'/*.nc',combine='nested',concat_dim='time')
        lifetime = dataset['Lifetime'][24*30:,2:62]
        tsc = dataset['TSC'][24*30:,2:62]
        oceanland = dataset['ConvOrigin'][24*30:,2:62]
        detinsit = dataset['ConvCir'][24*30:,2:62]

        lifetime_ya.append(np.asarray(np.nanmean(lifetime)))
        tsc_ya.append(np.asarray(np.nanmean(tsc)))

        landavg.append(np.nansum(np.asarray(np.where(oceanland>0,1,0))))
        oceanavg.append(np.nansum(np.asarray(np.where(oceanland<0,1,0))))

    except:
        lifetime_ya.append(np.nan)
        tsc_ya.append(np.nan)
        landavg.append(np.nan)
        oceanavg.append(np.nan)
    print(i)


In [ ]:
ratio = np.asarray(lifetime_ya)/np.asarray(tsc_ya)
lifetime_ya = np.asarray(lifetime_ya)

In [ ]:
tsc1992 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/1992/*.nc',combine='nested',concat_dim='time')['TSC'][24*31:,2:62]
tsc1985 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/1985/*.nc',combine='nested',concat_dim='time')['TSC'][24*31:,2:62]
lifetime1992 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/1992/*.nc',combine='nested',concat_dim='time')['Lifetime'][24*51:,2:62]
lifetime1985 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/1985/*.nc',combine='nested',concat_dim='time')['Lifetime'][24*51:,2:62]

In [ ]:

#plt.plot(np.nanmean(tsc1992[:],axis=(1,2)))
print(np.nanmean(lifetime1992[:]))
print(np.nanmean(lifetime1985[:]))

plt.plot(np.nanmean(tsc1992[:],axis=(1,2)))
plt.plot(np.nanmean(tsc1985[:],axis=(1,2)))

In [ ]:
plt.figure(dpi=500)
plt.imshow(np.nanmean(lifetime1985[:],axis=0) - np.nanmean(lifetime1992[:],axis=0),vmin=-50,vmax=50,cmap='RdBu_r',origin='lower')

In [ ]:
fig,ax1 = plt.subplots(dpi=500)
ax1.plot(lifetime_ya,'r',label='Lifetime')
ax2 = ax1.twinx()
#line2, = ax2.plot(tsc_ya,'b',label='TSC')
#ax3 = ax1.twinx()
ax2.plot(tsc_ya,'g',label='TSC')
#plt.legend([line1,line2],['Lifetime','TSC'])
plt.xlabel('Year')
plt.legend()
plt.xticks(np.arange(0,33,5),np.arange(1984,2017,5))


In [ ]:

yearlydcc = np.zeros((2016-1984))
yearlydcc_land = np.zeros((2016-1984))
yearlydcc_ocean = np.zeros((2016-1984))

fn2 = nc.Dataset('/net/seldon/disk2/Data/ECMWF/ERA5/SST_surf_1grid_timed/2020/SST_surf_001.nc')
sst = fn2['sst'][:]
maskl = np.where(sst[0]<0,0,1)

for i in range(1984,2017,1):
    try:
        print(i)
        dcc = xr.open_mfdataset('/disk1/Users/gah20/DCC/'+str(i)+'/*.nc',combine='nested',concat_dim='time')['DCC'][:,60:120]
        yearlydcc[i-1984] = np.nansum(dcc/dcc)

        dcc_l=np.where(maskl[60:120]==0,dcc,np.nan)
        dcc_o=np.where(maskl[60:120]==1,dcc,np.nan)

        yearlydcc_land[i-1984] = np.nansum(dcc_l/dcc_l)
        yearlydcc_ocean[i-1984] = np.nansum(dcc_o/dcc_o)
    except:
        yearlydcc[i-1984] = np.nan
        yearlydcc_land[i-1984] = np.nan
        yearlydcc_ocean[i-1984] = np.nan

In [ ]:
np.asarray(ctp_dcc)[0]

In [ ]:
plt.figure(dpi=500)
plt.stackplot(np.arange(0,32,1), yearlydcc_land, yearlydcc_ocean,colors =['orange', 'skyblue'],labels=['Land','Ocean'])
#plt.xlim(0,32)
#plt.xticks(ticks=np.arange(0,34,4),labels=np.arange(1984,2017,4),rotation=45)
plt.xlabel('Year')
plt.ylabel('DCC Frequency')
plt.legend()
#plt.savefig('ENSOSTUDY/DCC_LINEPLOT.pdf',bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(dpi=500)
plt.plot(yearlydcc/np.nanmax(yearlydcc),label='All',color='black')
plt.plot(yearlydcc_land/np.nanmax(yearlydcc),label='Land',color='orange')
plt.plot(yearlydcc_ocean/np.nanmax(yearlydcc),label='Ocean',color='skyblue')
plt.xlabel('Year')
plt.xticks(ticks=np.arange(0,34,2),labels=np.arange(1984,2017,2),rotation=45)
plt.ylabel('DCC Frequency')
plt.show()

## Import MEI ENSO INDEX DATA

In [ ]:
import pandas as pd
dates = ['Year','DJ','JF','FM','MA','AM','MJ','JJ','JA','AS','SO','ON','ND']
with open('/disk1/Users/gah20/meiv2.data.txt') as f:
    df = pd.DataFrame([line.strip().split() for line in f.readlines()][6:39])
    headers = [str(dates[i]) for i in range(0, 13)]
df.columns = headers
df['DJ'] = df['DJ'].astype(float)
for i in headers[1:]:
    df[i] = df[i].astype(float)

In [ ]:
df

In [ ]:
ensohist = []
plt.figure(dpi=500)
for year in range(1984,2017,1):
    for i in range(12):
        dcc = xr.open_mfdataset('/disk1/Users/gah20/DCC/'+str(year)+'/'+str(i+1).zfill(2)+'.nc',combine='nested',concat_dim='time')['DCC'][:,60:120]
        dccmonthly = np.nansum(dcc/dcc)
        enso = df[df['Year']==str(year)].to_numpy()[0][i+1]
        #ensohist.append(enso)
        #print(enso,dccmonthly)
        plt.scatter(enso,dccmonthly,alpha=0.5,color='k',s=5)
        print(str(year) + ' Done')
plt.xlabel('MEI ENSO Index')
plt.ylabel('DCC Frequency')
plt.savefig('ENSOSTUDY/ENSO_DCC_SCATTER.pdf',bbox_inches='tight',dpi=500)
plt.show()
    

In [ ]:
fn_cf = '/disk1/Users/gah20/DCC/*/*.nc'
ds_cf = xr.open_mfdataset(fn_cf,combine='nested',concat_dim='time')
ctp_dcc = ds_cf['DCC'][:,60:120]
lentime = ds_cf['Time']

In [ ]:

tsc2000 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2000/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2000 = np.nanmean(tsc2000[:],axis=0)
#vars()['tsc2008']
tsc2001 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2001/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2001 = np.nanmean(tsc2001[:],axis=0)

tsc2002 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2002/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2002 = np.nanmean(tsc2002[:],axis=0)

tsc2003 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2003/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2003 = np.nanmean(tsc2003[:],axis=0)

tsc2004 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2004/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2004 = np.nanmean(tsc2004[:],axis=0)

tsc2005 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2005/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2005 = np.nanmean(tsc2005[:],axis=0)

tsc2006 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2006/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2006 = np.nanmean(tsc2006[:],axis=0)

tsc2007 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2007/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2007 = np.nanmean(tsc2007[:],axis=0)

tsc2008 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2008/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2008 = np.nanmean(tsc2008[:],axis=0)

tsc2009 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2009/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2009 = np.nanmean(tsc2009[:],axis=0)

tsc2010 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2010/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2010 = np.nanmean(tsc2010[:],axis=0)

tsc2011 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2011/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2011 = np.nanmean(tsc2011[:],axis=0)

tsc2012 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2012/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2012 = np.nanmean(tsc2012[:],axis=0)

tsc2013 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2013/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2013 = np.nanmean(tsc2013[:],axis=0)

tsc2014 = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/2014/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
tscyearly2014 = np.nanmean(tsc2014[:],axis=0)

In [ ]:
hist2000 = np.asarray(tsc2000[24*100:]).flatten()
hist2001 = np.asarray(tsc2001[24*100:]).flatten()
hist2002 = np.asarray(tsc2002[24*100:]).flatten()
hist2003 = np.asarray(tsc2003[24*100:]).flatten()
hist2004 = np.asarray(tsc2004[24*100:]).flatten()
hist2007 = np.asarray(tsc2007[24*100:]).flatten()
hist2008 = np.asarray(tsc2008[24*100:]).flatten()
hist2009 = np.asarray(tsc2009[24*100:]).flatten()
hist2010 = np.asarray(tsc2010[24*100:]).flatten()
hist2011 = np.asarray(tsc2011[24*100:]).flatten()
hist2012 = np.asarray(tsc2012[24*100:]).flatten()
hist2013 = np.asarray(tsc2013[24*100:]).flatten()
hist2014 = np.asarray(tsc2014[24*100:]).flatten()

In [ ]:
plt.figure(dpi=500)
plt.hist(hist2000,bins=np.arange(0,800,10),histtype='step',label='2000')
plt.hist(hist2001,bins=np.arange(0,800,10),histtype='step',label='2001')
plt.hist(hist2002,bins=np.arange(0,800,10),histtype='step',label='2002')
plt.hist(hist2003,bins=np.arange(0,800,10),histtype='step',label='2003')
plt.hist(hist2004,bins=np.arange(0,800,10),histtype='step',label='2004')
plt.hist(hist2007,bins=np.arange(0,800,10),histtype='step',label='2007')
plt.hist(hist2008,bins=np.arange(0,800,10),histtype='step',label='2008')
plt.hist(hist2009,bins=np.arange(0,800,10),histtype='step',label='2009')
plt.hist(hist2010,bins=np.arange(0,800,10),histtype='step',label='2010')
plt.hist(hist2011,bins=np.arange(0,800,10),histtype='step',label='2011')
plt.hist(hist2012,bins=np.arange(0,800,10),histtype='step',label='2012')
plt.hist(hist2013,bins=np.arange(0,800,10),histtype='step',label='2013')
plt.hist(hist2014,bins=np.arange(0,800,10),histtype='step',label='2014')
plt.xlim(0,600)
plt.legend()
plt.xlabel('TSC / hours')
plt.ylabel('Frequency')
#plt.savefig('ENSOSTUDY/TSC_HIST.pdf',bbox_inches='tight',dpi=500)
plt.show()

In [ ]:
def round_three(n):
    '''round nearest integer to multiple of three'''
    rounded = np.int(3*np.round(n/3.0))
    if n<2:
        return rounded
    else:
        return int(rounded / 3)

In [ ]:
 tsc24 = []
 dcc24 = []
for year in range (1984,2017,1):
    i=0
    print('starting '+str(year))
    tscyear = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/'+str(year)+'/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
    dccyear = xr.open_mfdataset('/disk1/Users/gah20/DCC/'+str(year)+'/*.nc',combine='nested',concat_dim='time')['DCC'][:,60:120]
    for i in range(int(len(tscyear))):
        tsc24.append(np.nanmean(tscyear[i]))
        dcc24.append(np.nansum(np.nansum(dccyear[round_three(i)]/dccyear[round_three(i)],axis=0),axis=0))


In [ ]:
year=2001
i=1
dcc = xr.open_mfdataset('/disk1/Users/gah20/DCC/'+str(year)+'/'+str(i+1).zfill(2)+'.nc',combine='nested',concat_dim='time')['DCC'][:,60:120]
dccmonthly = np.nansum(dcc/dcc)

In [ ]:
enso.append(df[df['Year']==str(2011)].to_numpy()[0][i+1])

In [ ]:
enso = []
k=0
tscmonthly = []
for year in range(1984,2017,1):
        k=k+1
        print(year)
        start = 0
        for i in range(0,12):
                enso.append(df[df['Year']==str(year)].to_numpy()[0][i+1])
                #tscmonthly.append(np.nanmean(tsc[start*3:(start+len(dcc))*3]))
        

plt.figure(dpi=500,figsize=(10,3))
plt.bar(np.arange(0,(2017-1984)*12,1),enso,color=np.where(np.asarray(enso) < 0, 'r', 'b'),width=1,alpha=0.5,align='edge')
plt.xlabel('Year')
plt.grid(axis='x')
plt.ylabel('MEI_v2 ENSO Index')
plt.xticks(ticks=np.arange(0,(2017-1984)*12,12),labels=np.arange(1984,2017,1),rotation=45)
plt.xlim(2,(2017-1984)*12)
#plt.savefig('ENSOSTUDY/ENSO.pdf',bbox_inches='tight',dpi=500)

In [ ]:
year='2009'
start = 0
for i in range(0,12):
        dcc = xr.open_mfdataset('/disk1/Users/gah20/DCC/'+year+'/'+str(i+1).zfill(2)+'.nc',combine='nested',concat_dim='time')['DCC'][:,60:120]
        dccmonthly = np.nansum(dcc/dcc)
        start = start + len(dcc)
        print(start*3)

In [ ]:
ensohist = []
import traceback
start = 0
p_enso_tsc = np.zeros((60,360))
n_enso_tsc = np.zeros((60,360))

p_enso_dcc = np.zeros((60,360))
n_enso_dcc = np.zeros((60,360))

p_enso_dcccount = 0
n_enso_dcccount = 0

p_enso_tsccount = 0
n_enso_tsccount = 0
k=0
month=0
plt.figure(dpi=500)
colors = plt.cm.jet(np.linspace(0,1,32))
for year in range(1985,2017,1):
    if year == 1998:
        continue
    try:
        color = colors[k % len(colors)]
        k=k+1
        print(year)
        start = 0
        tsc = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/'+str(year)+'/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
        for i in range(0,11):
            dcc = xr.open_mfdataset('/disk1/Users/gah20/DCC/'+str(year)+'/'+str(i+1).zfill(2)+'.nc',combine='nested',concat_dim='time')['DCC'][:,60:120]
            dccmonthly = np.nansum(dcc/dcc)
            tscmonthly = np.nanmean(tsc[start*3:(start+len(dcc))*3,:,:])
            enso = df[df['Year']==str(year)].to_numpy()[0][i+2]
            if enso > 0.5 :
                p_enso_tsc = p_enso_tsc + np.nanmean(tsc[start*3:(start+len(dcc))*3],axis=0)
                p_enso_tsccount = p_enso_tsccount + 1

                p_enso_dcc = p_enso_dcc + np.nansum(dcc/dcc,axis=0)
                p_enso_dcccount = p_enso_dcccount + 1
            if enso < -0.5 :
                n_enso_tsc = n_enso_tsc + np.nanmean(tsc[start*3:(start+len(dcc))*3],axis=0)
                n_enso_count = n_enso_count + 1

                n_enso_dcc = n_enso_dcc + np.nansum(dcc/dcc,axis=0)
                n_enso_dcccount = n_enso_dcccount + 1
                if np.isnan(n_enso_tsc).any():
                    print('nan: '+str(i))
            #print('start month' + str(start*3))
            #print((start+len(dcc))*3)
            start = start + len(dcc)       
            
            plt.scatter(tscmonthly,enso,alpha=0.5,color=color,s=5)
            month = month+1
        print(str(year) + ' Done')                               
        #print(str(year) + ' Done')
        
        plt.scatter(tscmonthly,enso,alpha=0.5,color=color,s=5,label=str(year))
    except Exception as e:
        print(e)
plt.xlabel('TSC / hrs')
plt.ylabel('ENSO Index')
plt.xlim(150,200)
#plt.ylim(50000)
plt.legend()
#plt.savefig('ENSOSTUDY/TSC_ENSO_scatter.pdf',bbox_inches='tight',dpi=500)
plt.show()

In [ ]:
tscmont

In [ ]:
gs = fig.add_gridspec(3, 1)
ax0 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree(central_longitude=0))
ax0.set_extent([-180,180,-29.5,29.5],ccrs.PlateCarree(central_longitude=0))
ax0.coastlines(resolution='auto', color='k',linewidth=0.4)
plt.show()

In [ ]:
img_extent = (-180, 180, -30, 30)
fig = plt.figure(figsize=(10,6),dpi=500)
gs = fig.add_gridspec(3, 1)

ax0 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax0.set_extent([-180,180,-29.5,29.5],ccrs.PlateCarree())
ax0.coastlines(resolution='auto', color='k',linewidth=0.4)
ax1 = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax1.set_extent([-180,180,-29.5,29.5],ccrs.PlateCarree())
ax1.coastlines(resolution='auto', color='k',linewidth=0.4)
ax2 = fig.add_subplot(gs[2, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax2.set_extent([-180,180,-29.5,29.5],ccrs.PlateCarree())
ax2.coastlines(resolution='auto', color='k',linewidth=0.4)

im0=ax0.imshow(p_enso_dcc/p_enso_dcccount,origin='lower',extent=img_extent,transform=ccrs.PlateCarree(central_longitude=0),vmin=0,vmax=20,cmap='viridis',aspect='auto')
ax0.set_title('Positive ENSO')
plt.colorbar(im0,ax=ax0,orientation='vertical',label='TSC / hrs')

im1=ax1.imshow(n_enso_dcc/n_enso_dcccount,origin='lower',extent=img_extent,transform=ccrs.PlateCarree(central_longitude=0),vmin=0,vmax=20,cmap='viridis',aspect='auto')
ax1.set_title('Negative ENSO < -0.5')
plt.colorbar(im1,ax=ax1,orientation='vertical',label='TSC / hrs')


im2=ax2.imshow((p_enso_dcc/p_enso_dcccount) - (n_enso_dcc/n_enso_dcccount),origin='lower',extent=img_extent,transform=ccrs.PlateCarree(central_longitude=0),vmin=-15,vmax=15,cmap='RdBu_r',aspect='auto')
ax2.set_title('Positive ENSO - Negative ENSO')
plt.colorbar(im2,ax=ax2,orientation='vertical',label='TSC Diff / hrs')


In [ ]:
img_extent = (-180, 180, -30, 30)
fig = plt.figure(figsize=(10,6),dpi=500)
gs = fig.add_gridspec(3, 1)

ax0 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree())
ax0.set_extent([-180,180,-29.5,29.5],ccrs.PlateCarree())
ax0.coastlines(resolution='auto', color='k',linewidth=0.4)
ax1 = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree())
ax1.set_extent([-180,180,-29.5,29.5],ccrs.PlateCarree())
ax1.coastlines(resolution='auto', color='k',linewidth=0.4)
ax2 = fig.add_subplot(gs[2, 0], projection=ccrs.PlateCarree())
ax2.set_extent([-180,180,-29.5,29.5],ccrs.PlateCarree())
ax2.coastlines(resolution='auto', color='k',linewidth=0.4)

im0=ax0.imshow(p_enso_tsc/p_enso_count,origin='lower',extent=img_extent,transform=ccrs.PlateCarree(central_longitude=180),vmin=0,vmax=300,cmap='viridis',aspect='auto')
ax0.set_title('Positive ENSO')
plt.colorbar(im0,ax=ax0,orientation='vertical',label='TSC / hrs')

im1=ax1.imshow(n_enso_tsc/n_enso_count,origin='lower',extent=img_extent,transform=ccrs.PlateCarree(central_longitude=180),vmin=0,vmax=300,cmap='viridis',aspect='auto')
ax1.set_title('Negative ENSO < -0.5')
plt.colorbar(im1,ax=ax1,orientation='vertical',label='TSC / hrs')


im2=ax2.imshow(p_enso_tsc/p_enso_count - n_enso_tsc/n_enso_count,origin='lower',extent=img_extent,transform=ccrs.PlateCarree(central_longitude=180),vmin=-70,vmax=70,cmap='RdBu_r',aspect='auto')
ax2.set_title('Positive ENSO - Negative ENSO')
plt.colorbar(im2,ax=ax2,orientation='vertical',label='TSC Diff / hrs')

plt.savefig('ENSO_DCC_Map_EL.pdf',bbox_inches='tight',dpi=500)

plt.figure(dpi=500)
plt.imshow(TSC2001NEW[0,:,90:315]==0,origin='lower')
plt.figure(dpi=500)
plt.imshow(dcc2001[0]/dcc2001[0]==1,origin='lower')



In [ ]:
dailymeantsc = []
for i in range(len(totaltsc)):
    dailymeantsc.append(np.nanmean(totaltsc[(i*24*365):(i+1)*24*365]))
    if i % 100 == 0:
        print(i)

In [ ]:
enso=[]
for year in range(2000,2013,1):
    enso.append(((np.asarray(df[df['Year']==str(year)].to_numpy()[0][1:])).astype(float)).mean())
    

In [ ]:
plt.plot(dailymeantsc[0:13],enso[0:13],marker='.',linestyle='none')
plt.plot()

In [ ]:
len(dailymeantsc)

In [ ]:
tsclinreg = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/*/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]

In [ ]:
for i in range

In [ ]:
def slope(x):
    sl = stats.linregress(x.time.astype(float),x[dict(paired_points=0)]).slope
    return xr.DataArray(sl)

slope = np.zeros((60,360))
intercept = np.zeros((60,360))
for i in range(0,60,1):
    for j in range(0,360,1):
        x = tscyear[:,i,j]
        y = np.arange(0,len(x))
        reg = stats.linregress(y,x)
        slope[i,j] = reg.slope
        intercept[i,j] = reg.intercept
        if j % 10 == 0:
            print(j)

In [ ]:
slope = np.zeros((60,360))
intercept = np.zeros((60,360))
for i in range(0,60,1):
    for j in range(0,360,1):
        x = tscyear[:,i,j]
        y = np.arange(0,len(x))
        reg = stats.linregress(y,x)
        slope[i,j] = reg.slope
        intercept[i,j] = reg.intercept
        if j % 10 == 0:
            print(j)

In [ ]:
np.nanmean(slope)

In [ ]:

img_extent = (-180, 180, -30, 30)
fig = plt.figure(figsize=(10,6),dpi=500)
gs = fig.add_gridspec(1, 1)

ax0 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree())
ax0.set_extent([-180,180,-29.5,29.5],ccrs.PlateCarree())
ax0.coastlines(resolution='auto', color='k',linewidth=0.4)

plt.imshow(slope,origin='lower',vmin=-5,vmax=5,cmap='RdBu_r',extent=img_extent,transform=ccrs.PlateCarree(central_longitude=180))
plt.colorbar(orientation='vertical',label='Trend in TSC  / hrs per year')

In [ ]:
tscyear = np.zeros((15,60,360))
for year in range(2000,2015,1):
    tscyearly = xr.open_mfdataset('/disk1/Users/gah20/TSCnew/'+str(year)+'/*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
    tscyear[year-2000] = np.nanmean(np.asarray(tscyearly),axis=0)

In [ ]:
plt.imshow((tscyear - np.nanmean(tscyear,axis=0))[8],cmap='RdBu_r',origin='lower',vmax=50,vmin=-50)

In [ ]:
plt.figure(dpi=500)
im = plt.imshow(np.sqrt(np.nansum((tscyear - np.nanmean(tscyear,axis=0))**2,axis=0)/15),cmap='viridis',vmin=0)
plt.colorbar(im,orientation='horizontal',label='TSC std dev / hrs')

In [ ]:
for i in range(0,13,1):
    plt.figure(figsize=(18,3))
    plt.contourf(tscyear[i],levels=20,origin='lower',vmin=0,vmax=350,cmap='RdBu_r',aspect='equal')
    plt.title(str(i+2000))
    plt.show()

In [ ]:
tsc2000 = xr.open_mfdataset('/net/hardin/disk1/Users/gah20/TSCnew/200[0-9]/TSC_*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]

In [ ]:
oldair = np.nansum(np.where(tsc2000>500,1,np.nan),axis=(0))

In [ ]:

img_extent = (-180, 180, -30, 30)
fig = plt.figure(figsize=(10,6),dpi=500)
gs = fig.add_gridspec(1, 1)

ax0 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree())
ax0.set_extent([-180,180,-29.5,29.5],ccrs.PlateCarree())
ax0.coastlines(resolution='auto', color='white',linewidth=0.4)

plt.imshow(oldair,origin='lower',cmap='viridis',extent=img_extent,transform=ccrs.PlateCarree(central_longitude=180))
plt.colorbar(orientation='horizontal',label='OLD AIR / hrs')

In [ ]:
lifetime2001 = xr.open_mfdataset('/net/hardin/disk1/Users/gah20/TSCnew/200[0-9]/TSC_*.nc',combine='nested',concat_dim='time')['Lifetime'][:,2:62]

In [ ]:
lifetimetotal = []
for year in range(2000,2015,1): 
    lifetime = xr.open_mfdataset('/net/hardin/disk1/Users/gah20/TSCnew/'+str(year)+'/TSC_*.nc',combine='nested',concat_dim='time')['Lifetime'][:,2:62]
    lifetimetotal.append(np.nanmedian(lifetime,axis=(2,1)))

In [ ]:
plt.figure(dpi=500)
plt.plot(np.concatenate(lifetimetotal[:],axis=0)[5::500])
plt.ylim(20,80)

In [ ]:
#tscyearly = np.zeros((16,60,360))
tscyearlyplot = []
dccyearlyplot = []
lifetimeyearlyplot = []
detrainedyearly = []
insituyearly = []
tsc_land_yearlyplot = []
tsc_ocean_yearlyplot = []

for year in range(2000,2016,1):
    tsc = xr.open_mfdataset('/net/hardin/disk1/Users/gah20/TSCnew/'+str(year)+'/TSC_*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
    lifetime = xr.open_mfdataset('/net/hardin/disk1/Users/gah20/TSCnew/'+str(year)+'/TSC_*.nc',combine='nested',concat_dim='time')['Lifetime'][:,2:62]
    detins = xr.open_mfdataset('/net/hardin/disk1/Users/gah20/TSCnew/'+str(year)+'/TSC_*.nc',combine='nested',concat_dim='time')['ConvCir'][:,2:62]
    detrained = np.where(detins>0,tsc,np.nan)
    insitu = np.where(detins<0,tsc,np.nan)
    dcc = xr.open_mfdataset('/net/hardin/disk1/Users/gah20/DCC/'+str(year)+'/*.nc',combine='nested',concat_dim='time')['DCC'][:,60:120]
    tscyearly = np.nanmedian(tsc,axis=0)
    lifetimeyearlyplot.append(np.nanmedian(lifetime))
    detrainedyearly.append(np.nanmedian(detrained))
    insituyearly.append(np.nanmedian(insitu))
    #scyearly[i-2000] = np.nanmedian(tsc[:],axis=0)
    tscyearlyplot.append(np.nanmedian(tsc))

    dccyearlyplot.append(np.nansum(dcc/dcc))
    print(year)


In [ ]:
fig = plt.figure()
a1 = fig.add_axes([0,0,1,1])
x = np.arange(1,11)
a1.plot(tscyearlyplot, 'b-')
a1.set_ylabel('TSC / hour', color='b')
a2 = a1.twinx()
a2.set_ylabel('Detrained Cirrus Lifetime / hours', color='r')
a1.set_xlabel('Year')
a1.set_xticks(np.arange(0,16,2))
a1.set_xticklabels(np.arange(2000,2016,2),rotation=45)
a2.plot(detrainedyearly,'r-')
plt.savefig('ENSOSTUDY/TSC_LIFETIME_detrained.pdf',bbox_inches='tight',dpi=500)

In [ ]:
fig = plt.figure()
a1 = fig.add_axes([0,0,1,1])
x = np.arange(1,11)
a1.plot(tscyear, 'b-')
a1.set_ylabel('exp')
a2 = a1.twinx()
a2.plot(tsc_yearlyplot,'r-')

In [ ]:
wind1984 = np.load('/disk1/Users/gah20/Wind/U_1984_windmean.npy')


In [ ]:
plt.imshow(wind1984)
plt.show()